In [86]:
import queue as Q

import collections as C

import time

import resource

import sys

import math

import itertools

import heapq as H

#### SKELETON CODE ####

## The Class that Represents the Puzzle

d=time.time()

class PuzzleState(object):

    """docstring for PuzzleState"""

    def __init__(self, config, n, parent=None, action="Initial", cost=0):

        if n*n != len(config) or n < 2:

            raise Exception("the length of config is not correct!")

        self.n = n

        self.cost = cost

        self.parent = parent

        self.action = action

        self.dimension = n

        self.config = config

        self.children = []

        for i, item in enumerate(self.config):

            if item == 0:

                self.blank_row = int(i / self.n)

                self.blank_col = i % self.n

                break

    def display(self):

        for i in range(self.n):

            line = []

            offset = i * self.n

            for j in range(self.n):

                line.append(self.config[offset + j])

            print(line)

    def move_left(self):

        if self.blank_col == 0:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index - 1

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Left", cost=self.cost + 1)

    def move_right(self):

        if self.blank_col == self.n - 1:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index + 1

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Right", cost=self.cost + 1)

    def move_up(self):

        if self.blank_row == 0:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index - self.n
            
            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Up", cost=self.cost + 1)

    def move_down(self):

        if self.blank_row == self.n - 1:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index + self.n

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Down", cost=self.cost + 1)

    def expand(self):

        """expand the node"""

        # add child nodes in order of UDLR

        if len(self.children) == 0:

            up_child = self.move_up()

            if up_child is not None:

                self.children.append(up_child)

            down_child = self.move_down()

            if down_child is not None:

                self.children.append(down_child)

            left_child = self.move_left()

            if left_child is not None:

                self.children.append(left_child)

            right_child = self.move_right()

            if right_child is not None:

                self.children.append(right_child)

        return self.children
    
    def __eq__(self,other):
        return (self.config==other.config)
    
    def __lt__(self,other):
        return (self.config<other.config)
    
    def isGoal(self):
        return(self.config==tuple(range(self.n**2)))
    
    def manhattan(self):
        MD=0
        for i in range(self.n):
            for j in range(self.n):
                k=self.config[i*self.n+j]
                if k==0:
                    continue
                MD+=abs(k % self.n - j) + abs(int(k/self.n)-i)
        return(MD)

class Frontier(object):
    def __init__(self,initial_state,priority=0):
        self.heap=[]
        self.maxdepth=0
        self.nexpended=0
        self.states=C.deque()
        self.configs=dict()
        self.states.append(initial_state)
        self.counter = itertools.count()
        count = next(self.counter)
        self.configs[initial_state.config]=[priority,count]
        H.heappush(self.heap,[priority,count,initial_state])
        
        
    def append(self,state,priority=0):
        self.states.append(state)
        count = next(self.counter)
        self.configs[state.config]=[priority,count]
        H.heappush(self.heap,[priority,count,state])
        
    def update(self,state,priority=0):
        opriority,count=self.configs[state.config]
        if priority<opriority:
            self.configs[state.config]=[priority,count]
            #count = next(self.counter)
            H.heappush(self.heap,[priority,count,state])

    def pop(self):
        self.nexpended+=1
        ret = self.states.pop()
        self.configs.pop(ret.config)
        if ret.cost > self.maxdepth:
            self.maxdepth=ret.cost
        return ret

    def popleft(self):
        self.nexpended+=1
        ret = self.states.popleft()
        self.configs.pop(ret.config)
        if ret.cost > self.maxdepth:
            self.maxdepth=ret.cost
        return ret
    
    def poppriority(self):
        while self.heap:
            priority,count,state=H.heappop(self.heap)
            print(priority,count,state.config)
            opriority,ocount=self.configs[state.config]
            if opriority==priority:
                return(state)
        return None
    
    def not_empty(self):
        return self.states.not_empty()
    
    def length(self):
        return(len(self.states))

    
    
    def __contains__(self,state):
        return(state.config in self.configs)
    
    def maxx(self):
        maxd=0
        for st in self.states:
            if st.cost>maxd:
                maxd=st.cost
        return(maxd)

        
# Function that Writes to output.txt

### Students need to change the method to have the corresponding parameters

def writeOutput():
    pass

    ### Student Code Goes here

def bfs_search(initial_state):

    """BFS search"""
    
    frontier = Frontier(initial_state)
    expanded = set()
    i=0
    
    while frontier.not_empty:
        state = frontier.popleft()

        if state.isGoal():
            return(state,len(expanded),frontier.maxx())
        
        expanded.add(state.config)
        for neighbor in state.expand():
            if neighbor.config not in expanded:
                if neighbor not in frontier:
                    frontier.append(neighbor)
        

    ### STUDENT CODE GOES HERE ###

def dfs_search(initial_state):

    """DFS search"""
    
    frontier = Frontier(initial_state)
    expanded = set()
    i=0
    
    while frontier.not_empty:
        state = frontier.pop()

        if state.isGoal():
            return(state,len(expanded),frontier.maxx())
        
        expanded.add(state.config)
        for neighbor in reversed(state.expand()):
            if neighbor.config not in expanded:
                if neighbor not in frontier:
                    frontier.append(neighbor)
    
    
    ### STUDENT CODE GOES HERE ###

def A_star_search(initial_state):

    """A * search"""

    frontier = Frontier(initial_state)
    expanded = set()
    i=0
    
    while frontier.not_empty:
        state = frontier.poppriority()

        if state.isGoal():
            return(state,len(expanded),frontier.maxx())
        
        expanded.add(state.config)
        for neighbor in state.expand():
            if neighbor.config not in expanded:
                if neighbor not in frontier:
                    frontier.append(neighbor,1.001*neighbor.manhattan()+neighbor.cost)
                else:
                    frontier.update(neighbor,1.001*neighbor.manhattan()+neighbor.cost)
                    
                    
    ### STUDENT CODE GOES HERE ###

def calculate_total_cost(state):

    """calculate the total estimated cost of a state"""

    ### STUDENT CODE GOES HERE ###

def calculate_manhattan_dist(idx, value, n):

    """calculatet the manhattan distance of a tile"""

    ### STUDENT CODE GOES HERE ###

def test_goal(puzzle_state):

    """test the state is the goal state or not"""

    ### STUDENT CODE GOES HERE ###

# Main Function that reads in Input and Runs corresponding Algorithm

def main(sm,begin_states):

    # sm = sys.argv[1].lower()

    begin_state = begin_states.split(",")

    begin_state = tuple(map(int, begin_state))

    size = int(math.sqrt(len(begin_state)))

    hard_state = PuzzleState(begin_state, size)
    
    
    if sm == "bfs":

        return(bfs_search(hard_state))

    elif sm == "dfs":

        return(dfs_search(hard_state))

    elif sm == "ast":

        return(A_star_search(hard_state))

    else:

        print("Enter valid command arguments !")

d=time.time()
#st,lex,mm=main("bfs", "1,2,5,3,4,0,6,7,8")
#st,lex,mm=main("dfs","6,1,8,4,0,2,7,3,5")
#st,lex,mm=main("bfs","6,1,8,4,0,2,7,3,5")
#st,lex,mm=main("ast","6,1,8,4,0,2,7,3,5")
#st,lex,mm=main("dfs","8,6,4,2,1,3,5,7,0")
#st,lex,mm=main("bfs","8,6,4,2,1,3,5,7,0")
st,lex,mm=main("ast","8,6,4,2,1,3,5,7,0")
#print(time.time()-d)
path=[]
cost=st.cost
while st.parent is not None:
    #print(st.config,st.action,st.manhattan(),st.cost)
    path=[st.action]+path
    st=st.parent
print('path_to_goal:' , path)
print('cost_to_path:' , cost)
print('nodes_expanded:' , lex)
print('search_depth:' , cost)
print('max_search_depth:' , mm)
print('running_time:' , resource.getrusage(resource.RUSAGE_SELF).ru_utime+resource.getrusage(resource.RUSAGE_SELF).ru_stime)
print('max_ram_usage:', resource.getrusage(resource.RUSAGE_SELF).ru_maxrss*1e-6)


path_to_goal: ['Left', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Left', 'Up', 'Left']
cost_to_path: 26
nodes_expanded: 1668
search_depth: 26
max_search_depth: 26
running_time: 158.31529999999998
max_ram_usage: 0.573688


In [106]:
%run driver_3.py ast 8,6,4,2,1,3,5,7,0
%pfile output.txt

(8, 6, 4, 2, 1, 0, 5, 7, 3) 19
(8, 6, 4, 2, 1, 3, 5, 0, 7) 19
(8, 6, 0, 2, 1, 4, 5, 7, 3) 18
(8, 6, 4, 2, 0, 1, 5, 7, 3) 20
(8, 6, 4, 2, 0, 3, 5, 1, 7) 20
(8, 6, 4, 2, 1, 3, 0, 5, 7) 18
(8, 0, 6, 2, 1, 4, 5, 7, 3) 19
(8, 6, 4, 0, 1, 3, 2, 5, 7) 19
(8, 0, 4, 2, 6, 1, 5, 7, 3) 19
(8, 6, 4, 2, 7, 1, 5, 0, 3) 21
(8, 6, 4, 0, 2, 1, 5, 7, 3) 19
(8, 0, 4, 2, 6, 3, 5, 1, 7) 19
(8, 6, 4, 0, 2, 3, 5, 1, 7) 19
(8, 6, 4, 2, 3, 0, 5, 1, 7) 19
(8, 1, 6, 2, 0, 4, 5, 7, 3) 18
(0, 8, 6, 2, 1, 4, 5, 7, 3) 18
(0, 6, 4, 8, 1, 3, 2, 5, 7) 18
(8, 6, 4, 1, 0, 3, 2, 5, 7) 20
(0, 8, 4, 2, 6, 1, 5, 7, 3) 18
(8, 4, 0, 2, 6, 1, 5, 7, 3) 18
(0, 6, 4, 8, 2, 1, 5, 7, 3) 18
(8, 6, 4, 5, 2, 1, 0, 7, 3) 18
(0, 8, 4, 2, 6, 3, 5, 1, 7) 18
(8, 4, 0, 2, 6, 3, 5, 1, 7) 18
(0, 6, 4, 8, 2, 3, 5, 1, 7) 18
(8, 6, 4, 5, 2, 3, 0, 1, 7) 18
(8, 6, 0, 2, 3, 4, 5, 1, 7) 18
(8, 6, 4, 2, 3, 7, 5, 1, 0) 20
(8, 1, 6, 2, 7, 4, 5, 0, 3) 19
(8, 1, 6, 0, 2, 4, 5, 7, 3) 17
(8, 1, 6, 2, 4, 0, 5, 7, 3) 17
(2, 8, 6, 0, 1, 4, 5, 7, 3) 17
(6, 0, 4

(8, 6, 1, 4, 0, 2, 5, 7, 3) 16
(8, 1, 4, 2, 0, 3, 6, 5, 7) 14
(8, 4, 0, 2, 1, 3, 6, 5, 7) 14
(0, 3, 8, 2, 6, 4, 5, 1, 7) 16
(2, 3, 8, 5, 6, 4, 0, 1, 7) 14
(0, 6, 4, 1, 2, 3, 8, 5, 7) 16
(6, 4, 0, 1, 2, 3, 8, 5, 7) 14
(8, 6, 2, 5, 3, 4, 0, 1, 7) 14
(8, 6, 2, 3, 0, 4, 5, 1, 7) 14
(1, 5, 4, 6, 2, 3, 8, 0, 7) 13
(1, 5, 4, 0, 6, 3, 8, 2, 7) 15
(1, 5, 4, 6, 3, 0, 8, 2, 7) 13
(4, 0, 1, 2, 6, 8, 5, 7, 3) 15
(1, 0, 8, 2, 6, 4, 5, 7, 3) 15
(6, 0, 8, 1, 2, 4, 5, 7, 3) 15
(6, 2, 8, 1, 7, 4, 5, 0, 3) 15
(6, 2, 8, 1, 4, 0, 5, 7, 3) 13
(4, 8, 1, 6, 7, 2, 5, 0, 3) 15
(4, 8, 1, 0, 6, 2, 5, 7, 3) 15
(4, 8, 1, 6, 2, 0, 5, 7, 3) 15
(6, 0, 1, 4, 2, 8, 5, 7, 3) 13
(6, 2, 1, 4, 7, 8, 5, 0, 3) 13
(6, 2, 1, 4, 8, 0, 5, 7, 3) 13
(2, 0, 8, 6, 1, 3, 5, 4, 7) 13
(2, 1, 8, 0, 6, 3, 5, 4, 7) 13
(2, 1, 8, 6, 3, 0, 5, 4, 7) 11
(1, 8, 2, 6, 7, 4, 5, 0, 3) 13
(1, 8, 2, 0, 6, 4, 5, 7, 3) 13
(1, 8, 2, 6, 4, 0, 5, 7, 3) 11
(1, 2, 4, 6, 8, 0, 5, 7, 3) 13
(6, 0, 2, 8, 1, 3, 5, 4, 7) 13
(6, 1, 2, 0, 8, 3, 5, 4, 7) 11
(6, 1, 2

(6, 3, 2, 1, 5, 4, 0, 8, 7) 10
(6, 3, 2, 1, 5, 4, 8, 7, 0) 10
(6, 3, 0, 1, 4, 2, 8, 5, 7) 12
(6, 3, 2, 1, 4, 7, 8, 5, 0) 12
(1, 6, 2, 8, 3, 4, 0, 5, 7) 12
(1, 6, 2, 3, 0, 4, 8, 5, 7) 10
(3, 8, 2, 6, 1, 4, 0, 5, 7) 10
(3, 8, 2, 6, 1, 4, 5, 7, 0) 10
(3, 8, 0, 6, 4, 2, 5, 1, 7) 12
(3, 8, 2, 6, 4, 7, 5, 1, 0) 12
(6, 2, 1, 5, 4, 3, 8, 0, 7) 11
(4, 0, 2, 6, 1, 3, 5, 8, 7) 11
(4, 1, 2, 0, 6, 3, 5, 8, 7) 11
(4, 1, 2, 6, 3, 0, 5, 8, 7) 9
(4, 1, 2, 0, 8, 3, 6, 5, 7) 9
(6, 4, 2, 0, 1, 8, 5, 7, 3) 11
(4, 1, 2, 0, 7, 8, 6, 5, 3) 9
(4, 1, 2, 6, 7, 0, 5, 3, 8) 9
(1, 0, 2, 4, 6, 8, 5, 7, 3) 11
(4, 1, 2, 5, 6, 8, 7, 0, 3) 11
(4, 1, 2, 6, 5, 0, 7, 8, 3) 9
(2, 0, 8, 6, 1, 4, 3, 5, 7) 11
(6, 0, 2, 3, 1, 8, 5, 7, 4) 9
(2, 0, 1, 6, 5, 4, 8, 7, 3) 11
(2, 0, 4, 6, 1, 5, 8, 7, 3) 11
(2, 1, 4, 6, 7, 5, 8, 0, 3) 11
(2, 1, 4, 0, 6, 5, 8, 7, 3) 11
(6, 2, 1, 0, 3, 4, 8, 5, 7) 11
(1, 0, 4, 2, 8, 3, 5, 6, 7) 15
(1, 8, 4, 0, 2, 3, 5, 6, 7) 15
(1, 8, 4, 2, 3, 0, 5, 6, 7) 15
(1, 8, 4, 2, 6, 0, 5, 7, 3) 17
(1, 8, 4, 6, 0

(4, 0, 1, 8, 5, 2, 6, 7, 3) 11
(5, 2, 8, 1, 3, 0, 6, 7, 4) 11
(5, 0, 2, 3, 4, 8, 1, 6, 7) 9
(4, 1, 3, 0, 5, 2, 6, 8, 7) 9
(4, 1, 3, 6, 5, 0, 8, 7, 2) 11
(2, 0, 1, 6, 3, 4, 5, 7, 8) 9
(1, 3, 8, 5, 0, 4, 6, 2, 7) 12
(0, 1, 8, 5, 3, 4, 6, 2, 7) 10
(0, 3, 8, 1, 5, 4, 6, 2, 7) 12
(2, 4, 0, 3, 1, 8, 5, 7, 6) 10
(2, 4, 8, 3, 0, 1, 5, 7, 6) 12
(1, 0, 2, 5, 4, 8, 7, 3, 6) 9
(6, 1, 8, 0, 5, 4, 3, 2, 7) 11
(6, 1, 8, 3, 5, 0, 2, 7, 4) 11
(6, 0, 1, 3, 4, 8, 2, 5, 7) 11
(0, 1, 2, 3, 4, 7, 6, 8, 5) 4
(3, 1, 2, 4, 0, 7, 6, 8, 5) 6
(6, 1, 4, 0, 5, 2, 3, 8, 7) 9
(3, 1, 2, 0, 5, 8, 6, 7, 4) 5
(1, 4, 0, 3, 5, 2, 6, 7, 8) 4
(1, 4, 2, 3, 0, 5, 6, 7, 8) 2
(8, 4, 1, 5, 6, 3, 7, 0, 2) 15
(6, 4, 1, 8, 5, 0, 7, 2, 3) 15
(8, 6, 4, 0, 1, 3, 5, 7, 2) 17
(2, 8, 3, 5, 4, 6, 1, 0, 7) 17
(8, 0, 4, 2, 1, 3, 5, 7, 6) 17
(3, 6, 4, 8, 2, 0, 5, 1, 7) 17
(3, 2, 6, 8, 1, 4, 0, 5, 7) 14
(3, 2, 6, 8, 1, 4, 5, 7, 0) 14
(3, 2, 0, 8, 4, 6, 5, 1, 7) 14
(3, 2, 6, 8, 4, 7, 5, 1, 0) 16
(8, 1, 6, 0, 4, 3, 5, 7, 2) 15
(5, 2, 8, 0, 4, 6,

In [98]:
%run driver_3.py dfs 6,1,8,4,0,2,7,3,5
%pfile output.txt

Object `output.txt` not found.


In [100]:
%run driver_3.py bfs 6,1,8,4,0,2,7,3,5
%pfile output.txt

Object `output.txt` not found.


In [102]:
%run driver_3.py ast 6,1,8,4,0,2,7,3,5
%pfile output.txt

Object `output.txt` not found.


In [103]:
%run driver_3.py dfs 8,6,4,2,1,3,5,7,0
%pfile output.txt

Object `output.txt` not found.


In [104]:
%run driver_3.py bfs 8,6,4,2,1,3,5,7,0
%pfile output.txt

Object `output.txt` not found.


In [121]:
%run driver_3.py ast 8,6,4,2,1,3,5,7,0
%pfile output.txt

0 0 (8, 6, 4, 2, 1, 3, 5, 7, 0) 0 18
(8, 6, 4, 2, 1, 0, 5, 7, 3) 19 1
(8, 6, 4, 2, 1, 3, 5, 0, 7) 19 1
20.000018999999998 1 (8, 6, 4, 2, 1, 0, 5, 7, 3) 1 19
(8, 6, 0, 2, 1, 4, 5, 7, 3) 18 2
(8, 6, 4, 2, 0, 1, 5, 7, 3) 20 2
20.000017999999997 3 (8, 6, 0, 2, 1, 4, 5, 7, 3) 2 18
(8, 0, 6, 2, 1, 4, 5, 7, 3) 19 3
20.000018999999998 2 (8, 6, 4, 2, 1, 3, 5, 0, 7) 1 19
(8, 6, 4, 2, 0, 3, 5, 1, 7) 20 2
(8, 6, 4, 2, 1, 3, 0, 5, 7) 18 2
20.000017999999997 7 (8, 6, 4, 2, 1, 3, 0, 5, 7) 2 18
(8, 6, 4, 0, 1, 3, 2, 5, 7) 19 3
22.000018999999998 5 (8, 0, 6, 2, 1, 4, 5, 7, 3) 3 19
(8, 1, 6, 2, 0, 4, 5, 7, 3) 18 4
(0, 8, 6, 2, 1, 4, 5, 7, 3) 18 4
22.000017999999997 9 (8, 1, 6, 2, 0, 4, 5, 7, 3) 4 18
(8, 1, 6, 2, 7, 4, 5, 0, 3) 19 5
(8, 1, 6, 0, 2, 4, 5, 7, 3) 17 5
(8, 1, 6, 2, 4, 0, 5, 7, 3) 17 5
22.000017 12 (8, 1, 6, 0, 2, 4, 5, 7, 3) 5 17
(0, 1, 6, 8, 2, 4, 5, 7, 3) 16 6
(8, 1, 6, 5, 2, 4, 0, 7, 3) 16 6
22.000016 14 (0, 1, 6, 8, 2, 4, 5, 7, 3) 6 16
(1, 0, 6, 8, 2, 4, 5, 7, 3) 17 7
22.000016 15 (8, 1,

(0, 1, 8, 2, 6, 4, 5, 7, 3) 14 12
(2, 1, 8, 5, 6, 4, 0, 7, 3) 12 12
24.000011999999998 243 (2, 1, 8, 5, 6, 4, 0, 7, 3) 12 12
(2, 1, 8, 5, 6, 4, 7, 0, 3) 13 13
24.000013 84 (6, 2, 8, 0, 1, 4, 5, 7, 3) 11 13
(6, 2, 8, 5, 1, 4, 0, 7, 3) 12 12
(6, 2, 8, 1, 0, 4, 5, 7, 3) 14 12
24.000011999999998 245 (6, 2, 8, 5, 1, 4, 0, 7, 3) 12 12
(6, 2, 8, 5, 1, 4, 7, 0, 3) 13 13
24.000013 104 (6, 1, 2, 8, 7, 4, 5, 0, 3) 11 13
(6, 1, 2, 8, 7, 4, 0, 5, 3) 12 12
(6, 1, 2, 8, 7, 4, 5, 3, 0) 12 12
24.000011999999998 248 (6, 1, 2, 8, 7, 4, 0, 5, 3) 12 12
(6, 1, 2, 0, 7, 4, 8, 5, 3) 11 13
24.000011 250 (6, 1, 2, 0, 7, 4, 8, 5, 3) 13 11
(0, 1, 2, 6, 7, 4, 8, 5, 3) 10 14
(6, 1, 2, 7, 0, 4, 8, 5, 3) 12 14
24.00001 251 (0, 1, 2, 6, 7, 4, 8, 5, 3) 14 10
(1, 0, 2, 6, 7, 4, 8, 5, 3) 11 15
24.000011999999998 249 (6, 1, 2, 8, 7, 4, 5, 3, 0) 12 12
(6, 1, 2, 8, 7, 0, 5, 3, 4) 13 13
24.000013 125 (4, 0, 1, 6, 8, 2, 5, 7, 3) 11 13
(4, 8, 1, 6, 0, 2, 5, 7, 3) 14 12
(4, 1, 0, 6, 8, 2, 5, 7, 3) 12 12
24.000011999999998 256 (

(6, 1, 4, 3, 0, 5, 8, 7, 2) 8 16
(6, 1, 4, 3, 7, 5, 0, 8, 2) 8 16
24.000008 502 (6, 1, 4, 3, 0, 5, 8, 7, 2) 16 8
(6, 0, 4, 3, 1, 5, 8, 7, 2) 9 17
(6, 1, 4, 0, 3, 5, 8, 7, 2) 9 17
(6, 1, 4, 3, 5, 0, 8, 7, 2) 9 17
24.000008 503 (6, 1, 4, 3, 7, 5, 0, 8, 2) 16 8
(6, 1, 4, 0, 7, 5, 3, 8, 2) 9 17
24.000016 148 (2, 8, 4, 6, 3, 7, 5, 1, 0) 8 16
(2, 8, 4, 6, 3, 7, 5, 0, 1) 17 9
24.000016 168 (6, 2, 4, 8, 3, 7, 5, 1, 0) 8 16
(6, 2, 4, 8, 3, 7, 5, 0, 1) 17 9
24.000017 16 (1, 0, 6, 8, 2, 4, 5, 7, 3) 7 17
(1, 2, 6, 8, 0, 4, 5, 7, 3) 16 8
(1, 6, 0, 8, 2, 4, 5, 7, 3) 16 8
24.000016 510 (1, 2, 6, 8, 0, 4, 5, 7, 3) 8 16
(1, 2, 6, 8, 7, 4, 5, 0, 3) 17 9
(1, 2, 6, 0, 8, 4, 5, 7, 3) 15 9
(1, 2, 6, 8, 4, 0, 5, 7, 3) 15 9
24.000014999999998 513 (1, 2, 6, 0, 8, 4, 5, 7, 3) 9 15
(0, 2, 6, 1, 8, 4, 5, 7, 3) 16 10
(1, 2, 6, 5, 8, 4, 0, 7, 3) 14 10
24.000014 516 (1, 2, 6, 5, 8, 4, 0, 7, 3) 10 14
(1, 2, 6, 5, 8, 4, 7, 0, 3) 15 11
24.000014999999998 514 (1, 2, 6, 8, 4, 0, 5, 7, 3) 9 15
(1, 2, 0, 8, 4, 6, 5, 7, 3) 

(8, 6, 4, 0, 5, 3, 1, 2, 7) 19 7
24.000017999999997 729 (8, 6, 4, 1, 5, 3, 2, 7, 0) 6 18
(8, 6, 4, 1, 5, 0, 2, 7, 3) 19 7
24.000018999999998 707 (8, 6, 4, 1, 3, 0, 2, 5, 7) 5 19
(8, 6, 0, 1, 3, 4, 2, 5, 7) 18 6
(8, 6, 4, 1, 3, 7, 2, 5, 0) 20 6
24.000017999999997 732 (8, 6, 0, 1, 3, 4, 2, 5, 7) 6 18
(8, 0, 6, 1, 3, 4, 2, 5, 7) 19 7
24.00002 175 (8, 6, 4, 2, 3, 7, 5, 1, 0) 4 20
(8, 6, 4, 2, 3, 7, 5, 0, 1) 21 5
24.000020999999997 50 (8, 6, 4, 2, 7, 1, 5, 0, 3) 3 21
(8, 6, 4, 2, 7, 1, 0, 5, 3) 20 4
(8, 6, 4, 2, 7, 1, 5, 3, 0) 20 4
24.00002 736 (8, 6, 4, 2, 7, 1, 0, 5, 3) 4 20
(8, 6, 4, 0, 7, 1, 2, 5, 3) 21 5
24.00002 737 (8, 6, 4, 2, 7, 1, 5, 3, 0) 4 20
(8, 6, 4, 2, 7, 0, 5, 3, 1) 21 5
26.000005 348 (3, 1, 2, 0, 4, 8, 6, 5, 7) 21 5
(3, 1, 2, 6, 4, 8, 0, 5, 7) 6 22
(3, 1, 2, 4, 0, 8, 6, 5, 7) 6 22
26.000005 390 (1, 0, 2, 3, 5, 4, 6, 8, 7) 21 5
(1, 5, 2, 3, 0, 4, 6, 8, 7) 6 22
(1, 2, 0, 3, 5, 4, 6, 8, 7) 6 22
26.000006 346 (1, 4, 2, 3, 0, 8, 6, 5, 7) 20 6
(1, 4, 2, 3, 5, 8, 6, 0, 7) 5 21
(1,

In [117]:
['Up','Down','Left','Right'].index('Down')

1